In [2]:
!nvidia-smi

Tue Apr 27 13:26:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/Colab Notebooks/Progetti Personali/foto_ale

/content/gdrive/MyDrive/Colab Notebooks/Progetti Personali/foto_ale


In [4]:
pip install -r requirements.txt

     |████████████████████████████████| 100.2MB 31kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=d44013151fb563481696545bc8af3402a16470111f522f17d29cc26c211e6bd8
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [23]:
import face_recognition
import os
import re
from PIL import Image
import pickle
from tqdm.notebook import tqdm
import numpy as np
import cv2

path_original = "/content/gdrive/MyDrive/Progetti Personali/foto_ale/Foto Alessandro"
path_save = "/content/gdrive/MyDrive/Progetti Personali/foto_ale/baseline/crop"
path_embedding = "/content/gdrive/MyDrive/Progetti Personali/foto_ale/baseline/embedding"
image_pattern = r".*\.(jpg|jpeg|png)"

shape = 800

In [26]:
original_photo_path = [os.path.join(path_original, f) for f in os.listdir(path_original) if re.match(image_pattern, f, flags=re.I)]
baseline_embedding = []

i = 0
for i, image_path in tqdm(enumerate(original_photo_path)):

    image = np.array(Image.open(image_path))
    h, w, _ = image.shape
    
    max_size = max(h, w)
    scale = shape/max_size

    if scale > 1:
      new_h, new_w = int(h * scale), int(w * scale)

      image = cv2.resize(image, dsize=(new_w, new_h), interpolation=cv2.INTER_CUBIC)
    else:
      pass

    face_locations = face_recognition.face_locations(image, model = 'cnn')

    for j, face_location in enumerate(face_locations):
            top, right, bottom, left = face_location

            cropped_face = image[top:bottom, left:right]

            pil_image = Image.fromarray(cropped_face)
            pil_image.save(os.path.join(path_save, f'cropped_{i}_{j}.png'))

In [30]:
baseline_embedding = [os.path.join(path_save, f) for f in os.listdir(path_save)]
baseline_image_list = []

for i, image_path in tqdm(enumerate(baseline_embedding)):
  image = face_recognition.load_image_file(image_path)
  try:
    face_enc = face_recognition.face_encodings(image, model = 'large')[0]
    baseline_image_list.append(face_enc)
  except:
    pass


In [31]:
with open(os.path.join(path_embedding, 'embedding_list.pkl'), 'wb') as file:
    pickle.dump(baseline_image_list, file, protocol=pickle.HIGHEST_PROTOCOL)